# Introduction to AI Agents

### Ali Masri
### Machine Learning Engineer @ Ford Motor Company

### Let us Connect

- [www.linkedin.com/in/alimasri](https://www.linkedin.com/in/alimasri)
- [github.com/alimasri](http://github.com/alimasri)
- [www.alimasri.info](https://www.alimasri.info)

## What are Agents?

A Generative AI agent can be defined as an application that attempts to achieve a **goal** by **observing** the world and **acting upon it** using the **tools** that it has at its disposal. Agents are **autonomous** and can act **independently** of human intervention - [**Google Agents White Paper**](https://www.kaggle.com/whitepaper-agents)

![Agent Runtime](./assets/agentruntime.png)

## Keys to Building Agents

![Keys to Building Agents](./assets/agents.png)

## Agent Frameworks

- Low-level Orchestration
  - [LangGraph](https://langchain-ai.github.io/langgraph/)
- High-level
  - [AutoGen](https://www.microsoft.com/en-us/research/project/autogen/)
  - [PydanticAI](https://ai.pydantic.dev)
  - [CrewAI](https://www.crewai.com/)
  - [Agent Development Kit (ADK)](https://google.github.io/adk-docs/)
- Code Generation
  - [SmolAgents](https://github.com/huggingface/smolagents)
- No-Code
  - [n8n](https://n8n.io/)
  - [CopilotStudio](https://www.microsoft.com/en-us/microsoft-copilot/microsoft-copilot-studio)
  - [Rivet](https://rivet.ironcladapp.com/)

## Hands-on Session

### Setting Up the Environment

1. Install uv <https://docs.astral.sh/uv/getting-started/installation/>
   1. Mac and Linux:
      ```bash
      curl -LsSf https://astral.sh/uv/install.sh | sh
      ```
   2. Windows:
      ```bash
      powershell -ExecutionPolicy ByPass -c "irm https://astral.sh/uv/install.ps1 | iex"
      ```
2. Install depencenies:
    ```bash
    uv sync
    ```
3. Setup environment variables:
   1. Make a copy of `.env.example` and rename it to `.env_vars`
      ```bash
      cp .env.example .env_vars
      ```
   2. Fill in the values for the environment variables in the `.env_vars` file

In [ ]:
from dotenv import load_dotenv

load_dotenv(".env_vars")

### Large Language Models

#### Connecting to an LLM

In [2]:
from langchain_openai import ChatOpenAI

DEFAULT_MODEL = "gpt-4o-mini"

llm = ChatOpenAI(model=DEFAULT_MODEL)

#### Simple Interaction

In [4]:
PROMPT = "Write one sentence about the Sun"

In [ ]:
llm.invoke(PROMPT).pretty_print()

#### Role Playing

In [ ]:
ROLE = "You are a 5 years old kid."
llm.invoke(f"{ROLE} {PROMPT}").pretty_print()

In [ ]:
llm.invoke(
    input=[
        ("system", ROLE),
        ("human", PROMPT),
    ]
).pretty_print()

#### Knowledge Cutoff

In [ ]:
from IPython.display import Markdown

# Real-time Information - FAIL
Markdown(llm.invoke("What are the latest AI models from OpenAI").content)

#### Hallucination

In [ ]:
a = 129012291
b = 12912
math_problem = f"What is the value of {a} x {b}?"
print(math_problem)

In [ ]:
Markdown(llm.invoke(math_problem).content)

In [ ]:
print(f"{a * b:,}")

### Tool Calling

<https://python.langchain.com/docs/concepts/tool_calling/>

#### Tools Definition

In [11]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers

    Args:
        a: The first number
        b: The second number

    Returns:
        int: The product of the two numbers
    """
    return a * b


@tool
def get_weather(city: str) -> str:
    """Get the weather of a city

    Args:
        city (str): The city to get the weather of

    Returns:
        str: The weather of the city
    """
    import requests

    return requests.get(f"https://wttr.in/{city}").text

#### Tool Binding

In [12]:
llm_with_tool = llm.bind_tools(tools=[multiply, get_weather])

#### Tool Calling Internals

In [ ]:
math_problem

In [14]:
messages = [math_problem]

ai_response = llm_with_tool.invoke(math_problem)
messages.append(ai_response)

In [ ]:
ai_response.content

In [ ]:
ai_response

In [ ]:
ai_response.tool_calls

![Tool Calling](./assets/tool_calling.png)

In [ ]:
tool_response = multiply.invoke(ai_response.tool_calls[0])
messages.append(tool_response)
tool_response.pretty_print()

In [ ]:
ai_response = llm_with_tool.invoke(messages)
messages.append(ai_response)
ai_response.pretty_print()

In [ ]:
messages

#### Multiple Tool Calls

In [ ]:
ai_response = llm_with_tool.invoke("What is the weather in Beirut? Multiply that by 3")
ai_response.pretty_print()

In [ ]:
messages = ["What is the weather in Beirut? Multiply that by 3"]
tools = {"multiply": multiply, "get_weather": get_weather}
while True:
    ai_response = llm_with_tool.invoke(messages)
    ai_response.pretty_print()
    messages.append(ai_response)
    if not ai_response.tool_calls:
        break
    for tool_call in ai_response.tool_calls:
        tool_response = tools[tool_call["name"]].invoke(tool_call)
        messages.append(tool_response)
        tool_response.pretty_print()

## Agentic Design Patterns

![Agentic Design Patterns](assets/multi-agent-architectures.png)

### ReAct Agent

![ReAct Agent](assets/react.png)

#### Creating the Agent

In [23]:
# https://langchain-ai.github.io/langgraph/prebuilt/#available-libraries
from langgraph.prebuilt import create_react_agent

In [ ]:
react_agent = create_react_agent(DEFAULT_MODEL, tools=[multiply, get_weather])
react_agent

#### Running the Agent

In [ ]:
agent_response = react_agent.invoke({"messages": ["What is the value of 129012291 x 12912"]})
for message in agent_response["messages"]:
    message.pretty_print()

In [ ]:
# The agent can handle multiple tool calls
response = react_agent.invoke(
    {"messages": ["Get today's weather in Beirut and multiply that by 3"]}
)
for _ in response["messages"]:
    _.pretty_print()

In [ ]:
response = react_agent.invoke(
    {"messages": ["I live in Beirut, is today a good day for swimming in my pool?"]}
)
for _ in response["messages"]:
    _.pretty_print()

#### Sales Agent Demo

```bash
uv run code/sales.py
```

### CodeAgents

#### Motivation

In [ ]:
rs = "rr1rrr2rrrr32rrrr121212rrrr1221"
rs.count("r")

In [ ]:
llm.invoke(f"How many 'r's are there in `{rs}`?").pretty_print()

https://platform.openai.com/tokenizer

In [ ]:
llm.invoke(
    f"Write a python code to count the number of 'r's in `{rs}`? Don't answer the question, just write the code."
).pretty_print()

In [ ]:
# https://github.com/huggingface/smolagents
from smolagents import CodeAgent, OpenAIServerModel

model = OpenAIServerModel(model_id="gpt-4o-mini")
codeagent = CodeAgent(model=model, tools=[])

In [ ]:
codeagent.run(f"How many 'r's are there in `{rs}`?")

![CodeAgents](assets/codeagent.png)

#### Real World Example

In [33]:
text = """\
Hello there, my name is John Doe. I live in Dearborn, Michigan. My phone number is 313-555-1212.
"""

problem = f"""\
Identify all PII information in the given text and return a list of:  
- The PII category.  
- The PII value.  
- The starting index of the PII information in the string.  
- The ending index of the PII information in the string.

Note that indexes are 0-based.

Text:
{text}
"""

In [ ]:
Markdown(ChatOpenAI(model=DEFAULT_MODEL).invoke(problem).content)

In [ ]:
text[18:27], text[34:53], text[61:76]

In [ ]:
codeagent.run(problem)

In [ ]:
text[24:32], text[44:62], text[83:95]

#### CodeAgents with Tools

In [38]:
from smolagents import CodeAgent, DuckDuckGoSearchTool

search_tool = DuckDuckGoSearchTool()
agent = CodeAgent(model=model, tools=[search_tool])

In [ ]:
question = (
    "What is the USD/EUR exchange rate is the last 5 days? Report the min, max, and average values"
)
agent.run(question)

### Supervisor Agent

![Supervisor](assets/langgraph-supervisor.png)

In [40]:
supervisor = ChatOpenAI(model=DEFAULT_MODEL)

instructions = """\
You are a supervisor for a team of agents.
Given the following context, you need to assign the question to the right agent.

Available agents:

- `chef`: A chef that can answer cooking questions
- `doctor`: A doctor that can answer medical questions
- `programmer`: An engineer that can answer technical questions

If the user question is answered already by the agent, you can say `END`.

<Context>
{context}
</Context>
"""

In [ ]:
supervisor.invoke(
    input=instructions.format(context="What are the main causes of migraine?")
).content

In [42]:
from typing import Literal

from pydantic import BaseModel, Field


class ResponseFormat(BaseModel):
    agent_name: Literal["chef", "doctor", "programmer", "END"] = Field(
        ..., title="The name of the agent that should handle the message"
    )


supervisor = llm.with_structured_output(ResponseFormat, method="function_calling")

In [ ]:
supervisor.invoke(input=instructions.format(context="What are the main causes of migraine?"))

Let us use the LangGraph Supervisor implementation <https://github.com/langchain-ai/langgraph-supervisor-py>

In [44]:
chef = create_react_agent(
    name="chef",
    model=ChatOpenAI(model=DEFAULT_MODEL),
    tools=[],
    prompt="You are a Michelin-starred chef that can provide recipes and cooking tips. Check the user's question and answer it accordingly.",
)

doctor = create_react_agent(
    name="doctor",
    model=ChatOpenAI(model=DEFAULT_MODEL),
    tools=[],
    prompt="You are a licensed physician that can provide medical advice. Check the user's question and answer it accordingly.",
)

programmer = create_react_agent(
    name="programmer",
    model=ChatOpenAI(model=DEFAULT_MODEL),
    tools=[],
    prompt="You are a senior software engineer that can help with coding problems. Check the user's question and answer it accordingly.",
)

In [45]:
from langgraph_supervisor import create_supervisor

supervisor_builder = create_supervisor(
    prompt="""\
You are a supervisor managing a team of agents.  

Your task is answer the user's question, by planning the right sequence of questions to ask the agents.
  
- **Do not attempt to answer the question yourself**—your role is strictly to delegate.  
- **Ensure you only assign questions within an agent's area of expertise.** Avoid giving tasks they are not qualified for.  
- **Review all agent responses** and compile the final answer into a clear and accurate report.
""",
    agents=[chef, doctor, programmer],
    model=ChatOpenAI(model=DEFAULT_MODEL),
)

supervisor = supervisor_builder.compile()

In [ ]:
message = "What are the main causes of migraine?"
response = supervisor.invoke({"messages": [message]})

for _ in response["messages"]:
    _.pretty_print()

In [ ]:
message = """\
Is it safe for a diabetic to eat caramelized chicken wings? Check the ingredients first, and then answer the question.
"""
response = supervisor.invoke({"messages": [message]})

for _ in response["messages"]:
    _.pretty_print()

## Swarm

![Swarm](assets/langgraph-swarm.png)

<https://github.com/langchain-ai/langgraph-swarm-py>

## Plan and Execute


![Plan and Execute](assets/plan-and-execute.png)

<https://langchain-ai.github.io/langgraph/tutorials/plan-and-execute/plan-and-execute/>

### Reasoning Without Observation (ReWOO)

![ReWOO](assets/rewoo.png)

```text
Plan: I need to know the teams playing in the superbowl this year
E1: Search[Who is competing in the superbowl?]
Plan: I need to know the quarterbacks for each team
E2: LLM[Quarterback for the first team of #E1]
Plan: I need to know the quarterbacks for each team
E3: LLM[Quarter back for the second team of #E1]
Plan: I need to look up stats for the first quarterback
E4: Search[Stats for #E2]
Plan: I need to look up stats for the second quarterback
E5: Search[Stats for #E3]
```
<https://langchain-ai.github.io/langgraph/tutorials/rewoo/rewoo/>


## LLM Tracing

- LLM tracing is the practice of tracking and understanding the **step-by-step** decision-making and thought processes within LLMs as they generate responses.
- Tracing helps to **debug** and **improve** the performance of LLMs.
- **Available tools**
  - [Langsmith](https://www.langchain.com/langsmith)
  - [Arize Phoenix](https://phoenix.arize.com/)
  - [Langfuse](https://langfuse.com)

![Langsmith](assets/langsmith.png)